# Soccer Dataset

Importing the libraries

In [15]:
import pandas as pd
import pymongo

from time import mktime
from dateutil import parser
from time import strptime
from pymongo import MongoClient
from tqdm import tqdm

## Creating New Features and Add Them to a Collection in MongoDB

Now we want to first create new features.
The first thing is to connect to a pymongo database.

In [16]:
client = MongoClient(host='10.101.24.100')
db = client.soccerdb

Below we define a function which we need it for the rest.
It calculates the number of a palticular event for an specific match and team

In [17]:
def count_some_event(event, matchId=None, teamId=None, playerId=None, extra_cond={}):
    cond = {'eventName': event}
    cond.update(extra_cond)
    
    if matchId is not None:
        cond.update({'matchId': matchId})
    if teamId is not None:
        cond.update({'teamId': teamId})
    if playerId is not None:
        cond.update('playerId')
        
        
    return db.events.count_documents(cond)

Also we need the set of all matches and the corresponding teams to work with.

In [18]:
set_matches2teams = set(map(lambda dic: (dic['matchId'], dic['teamId']), db.events.find(
    {'eventName': 'Pass', 'matchPeriod': '1H'}, 
    {'_id': 0, 'matchId': 1, 'teamId': 1})))

Now It is time to creating features

### 1. Count the number of each event in the first half of the game

First we should get the list of all events. In order to do this we find 3 different matchIds to find the set of events inside them. This helps decreasing the probability of not seeing an event due to it's absence in a random game.

In [19]:
some_matchids = [x['matchId'] for x in db.events.find({}, {'_id': 0, 'matchId': 1}, limit=3)]
set_event_names = set([x['eventName'] for x in db.events.find({
    'matchId': {'$in': some_matchids}}, {'_id': 0, 'eventName': 1})])

Now we count and add the number of each event of the match for each team

In [20]:
for matchId, teamId in set_matches2teams:
    base = {
        'matchId': matchId,
        'teamId': teamId
    }
    for event in set_event_names:
        db.results.update_one(base, {
            '$set': {
                'num{}'.format(
                    event.title().replace(' ', '')
                ): count_some_event(event, matchId, teamId, extra_cond={'matchPeriod': '1H'})
            }
        }, upsert=True)

### 2. Are they playing at their home?

If true, the team is playing at their own home. This might be correlated to the winning because of the weather condition, time zone, being exhausted or emotional state of the players.

In [21]:
for matchId, teamId in tqdm(set_matches2teams):
    isHome = db.matches.find_one({'wyId': matchId},
                                 {'_id': 0, 'teamsData.{}.side'.format(str(teamId)): 1}
                                )['teamsData'][str(teamId)]['side']
    isHome = (isHome == 'home')
    db.results.update_one({'matchId': matchId, 'teamId': teamId}, {'$set': {'isHome': isHome}})

100%|██████████| 102/102 [00:01<00:00, 96.32it/s]


### 3. Percentage of Accurate Passes
Being accurate when passing can be an indication of how good a team is performing during the game. This can later on be examined in moving windows.

In [22]:
for match_id, team_id in tqdm(list(set_matches2teams)):
    num_pass = count_some_event('Pass', match_id, team_id, extra_cond={'matchPeriod': '1H'})
    num_accurate_pass = count_some_event('Pass', match_id, team_id, extra_cond={
        'matchPeriod': '1H',
        'tags.id': 1801
    })
    rate = num_accurate_pass / num_pass
    db.results.update_one({'matchId': match_id, 'teamId': team_id}, {
        '$set': {'numAccuratePass': num_accurate_pass, 'rateAccuratePass': rate}
    })

100%|██████████| 102/102 [00:17<00:00,  5.89it/s]


### 4. Mean score
We find the mean scores before specific match for every team. There are 4 scores which we describe below: 
- score: the number of goals scored by the team during the match (not counting penalties); 
- scoreET: the number of goals scored by the team during the match, including the extra time (not counting penalties); 
- scoreHT: the number of goals scored by the team during the first half of the match; 
- scoreP: the total number of goals scored by the team after the penalties;

For computing these values we first add a timestamp field to matches documents to make it easy to find the matches before an specific match

In [23]:
for doc in db.matches.find({}, {'wyId': 1, 'date': 1}):
    match_id = doc['wyId']
    date = doc['date']
    date = int(mktime(parser.parse(date).timetuple()))
    db.matches.update_one({'wyId': match_id}, {'$set': {'timestamp': date}})

Now we find and add the mean of scores to the result collection

In [24]:
for match, team in set_matches2teams:
    game_time = db.matches.find_one({'wyId': match}, {'_id': 0, 'timestamp': 1})['timestamp']
    previous_scores_json = list(db.matches.find({
        'teamsData.{}'.format(str(team)): {'$exists': True},
        'timestamp': {'$lt': game_time}
    },
    {'_id': 0, 'teamsData.{}.score'.format(str(team)): 1, 
     'teamsData.{}.scoreET'.format(str(team)): 1,
     'teamsData.{}.scoreHT'.format(str(team)): 1,
     'teamsData.{}.scoreP'.format(str(team)): 1}))
    
    scores = [x['teamsData'][str(team)]['score'] for x in previous_scores_json]
    scoresET = [x['teamsData'][str(team)]['scoreET'] for x in previous_scores_json]
    scoresHT = [x['teamsData'][str(team)]['scoreHT'] for x in previous_scores_json]
    scoresP = [x['teamsData'][str(team)]['scoreP'] for x in previous_scores_json]
    
    if scores:
        mean_score = sum(scores) / len(scores)
    else:
        mean_score = 0
    if scoresET:
        mean_scoreET = sum(scoresET) / len(scoresET)
    else:
        mean_scoreET = 0
    if scoresHT:
        mean_scoreHT = sum(scoresHT) / len(scoresHT)
    else:
        mean_scoreHT = 0
    if scoresP:
        mean_scoreP = sum(scoresP) / len(scoresP)
    else:
        mean_scoreP = 0
    
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPrevScore': mean_score}})
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPrevScoreET': mean_scoreET}})
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPrevScoreHT': mean_scoreHT}})
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPrevScoreP': mean_scoreP}})
    

### 5. Mean of players' potential to each team in a specific match
First we read the data from the other new dataset

In [25]:
data_players = pd.read_csv("../data/fifa.csv")
data_players.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


Now add the potential and overall of the new dataset to the players collection in our dataset

In [26]:
for i in tqdm(range(data_players.shape[0])):
    db.players.update_one({'shortName': data_players['Name'].iloc[i]}, {'$set': {'overall': int(data_players['Overall'].iloc[i])}})
    db.players.update_one({'shortName': data_players['Name'].iloc[i]}, {'$set': {'potential': int(data_players['Potential'].iloc[i])}})

100%|██████████| 18207/18207 [06:39<00:00, 45.62it/s]


Now we find the mean of players quality of each team for every game

In [27]:
for match, team in tqdm(set_matches2teams):
    all_overalls = []
    all_potentials = []
    players = list(db.matches.find({
        'teamsData.{}'.format(str(team)): {'$exists': True}
    },{'_id': 0, 'teamsData.{}.formation.bench.playerId'.format(str(team)): 1, 
       'teamsData.{}.formation.lineup.playerId'.format(str(team)): 1 }))
    playerIdBench = [y['playerId'] for x in players for y in x['teamsData'][str(team)]['formation']['bench']]
    playerIdLineup = [y['playerId'] for x in players for y in x['teamsData'][str(team)]['formation']['lineup']]
    playerId = list(set(playerIdBench + playerIdLineup))
    for player in playerId:
        all_overalls.append(list(db.players.find(
            {'wyId' : player},
            {'_id': 0, 'overall': 1}))[0].get('overall', 0))
        all_potentials.append(list(db.players.find(
            {'wyId' : player},
            {'_id': 0, 'potential': 1}))[0].get('potential', 0))
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPlayerOverall': sum(all_overalls)/len(all_overalls)}})
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPlayerPotential': sum(all_potentials)/len(all_potentials)}})

100%|██████████| 102/102 [00:55<00:00,  1.85it/s]


### 6. Number of yellow cards, red cards and second yellow cards of the first half

In [28]:
for match_id, team_id in tqdm(list(set_matches2teams)):
    num_red_card = count_some_event('Foul', match_id, team_id, extra_cond={'matchPeriod': '1H',
                                                                      'tags.id': 1701})
    num_yellow_card = count_some_event('Foul', match_id, team_id, extra_cond={'matchPeriod': '1H',
                                                                      'tags.id': 1702})
    num_second_yellow_card = count_some_event('Foul', match_id, team_id, extra_cond={'matchPeriod': '1H',
                                                                      'tags.id': 1703})
    db.results.update_one({'matchId': match_id, 'teamId': team_id}, {
        '$set': {'numSecondYellowCard': num_second_yellow_card}})
    db.results.update_one({'matchId': match_id, 'teamId': team_id}, {
        '$set': {'numYellowCard': num_yellow_card}})
    db.results.update_one({'matchId': match_id, 'teamId': team_id}, {
        '$set': {'numRedCard': num_red_card}})

100%|██████████| 102/102 [00:23<00:00,  4.26it/s]


### 7. Number of goals in the first half

In [29]:
for match, team in set_matches2teams:
    goals_on_the_first_half_json = list(db.matches.find({'wyId': match,
        'teamsData.{}'.format(str(team)): {'$exists': True}
    },
    {'_id': 0, 'teamsData.{}.scoreHT'.format(str(team)): 1}))
    number_of_goals_in_the_first_half = [x['teamsData'][str(team)]['scoreHT'] for x in goals_on_the_first_half_json]
    #print(number_of_goals_in_the_first_half[0])
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'numGoalsFristHalf': number_of_goals_in_the_first_half[0]}})
    

### 8. Ball possesion

In [30]:
interestingEvNames = ['Pass', 'Duel', 'Free Kick',  'Others on the ball', 'Shot']
def add_percBallInfo (res_col, ev_col):
    mt_ids_in_res = [e['matchId'] for e in res_col.find({}, {'_id':0, 'matchId':1})]
    for mId in tqdm(mt_ids_in_res):
        tm_ids_in_match = [e['teamId'] for e in res_col.find({'matchId': mId}, {'_id':0, 'teamId':1})]
        all_mt_ev = [e for e in ev_col.find( { 'matchId': int(mId) } ) ]
        
        for tId in tm_ids_in_match:
            all_tm_ev = [e for e in all_mt_ev if e['teamId'] == int(tId)]
            interesting_ev_cnt = len([e for e in all_tm_ev if e['eventName'] in interestingEvNames])
            oppHalfField_ev_cnt = len([e for e in all_tm_ev if e['positions'][0]['x']>50])

            percBallPoss = interesting_ev_cnt/len(all_mt_ev) * 100
            percOppHalfField = oppHalfField_ev_cnt/len(all_mt_ev) * 100
            
            res_col.update_one({'matchId': mId, 'teamId': tId}, 
                               { '$set': 
                                {'percBallPoss': percBallPoss, 'percOppHalfField': percOppHalfField }
                               })

### 9. Class Variables
We have two class variables: 
- weather the team has won the game or not
- Will the team goal in the second half or not

Let's start with the first one:

In [31]:
for match_id, team_id in set_matches2teams:
    results = db.matches.find_one({
        'wyId': match_id,
    }, {
        'winner': 1,
        'teamsData.{}.score'.format(str(team_id)): 1, 
        'teamsData.{}.scoreET'.format(str(team_id)): 1,
        'teamsData.{}.scoreP'.format(str(team_id)): 1,
        'teamsData.{}.scoreHT'.format(str(team_id)): 1,
        '_id': 0
    })
    final_score = sum(results['teamsData'][str(team_id)].values())
    second_half_score = results['teamsData'][str(team_id)]['score']
    #Since we added twice
    final_score -= results['teamsData'][str(team_id)]['scoreHT'] 
    #Since we just want the number of goals in the second half of the game
    second_half_score -= results['teamsData'][str(team_id)]['scoreHT']
    is_winner = int(results['winner'] == team_id)
    
    db.results.update_one({'matchId': match_id, 'teamId': team_id}, {
        '$set': {
            'finalScore': final_score,
            'numSecondHalfGoals': second_half_score,
            'isWinner': is_winner
        }
    })

Now the second one:

In [32]:
for match, team in set_matches2teams:
    number_of_goals_in_the_second_half_json = list(db.results.find({'matchId': match,
        'teamId': team
    },{'_id': 0, 'numSecondHalfGoals': 1}))
    number_of_goals_in_the_second_half = [x['numSecondHalfGoals'] for x in number_of_goals_in_the_second_half_json]
    if (number_of_goals_in_the_second_half[0] > 0):
        target = 1
    else:
        target = 0
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'didScoreInSecondHalf': target}})

So this is the final result collection:

In [33]:
db.results.find_one()

{'_id': ObjectId('5da399af04f673bbf273049a'),
 'didScoreInSecondHalf': 1,
 'finalScore': 3,
 'isHome': True,
 'isWinner': 1,
 'matchId': 1694430,
 'meanPlayerOverall': 58.21739130434783,
 'meanPlayerPotential': 59.47826086956522,
 'meanPrevScore': 1.0,
 'meanPrevScoreET': 0.0,
 'meanPrevScoreHT': 0.6666666666666666,
 'meanPrevScoreP': 0.0,
 'numAccuratePass': 259,
 'numDuel': 108,
 'numFoul': 7,
 'numFreeKick': 16,
 'numGoalkeeperLeavingLine': 0,
 'numGoalsFristHalf': 2,
 'numInterruption': 0,
 'numOffside': 0,
 'numOthersOnTheBall': 14,
 'numPass': 294,
 'numRedCard': 0,
 'numSaveAttempt': 1,
 'numSecondHalfGoals': 1,
 'numSecondYellowCard': 0,
 'numShot': 13,
 'numYellowCard': 0,
 'rateAccuratePass': 0.8809523809523809,
 'teamId': 3148}

Now it is time to visualize the dataset and understand which featurs are important